# Trabalhando bem

Vamos abordar alguns aspectos de como fazer um algoritmo/pacote
nos padrões esperados do Julia, e também um workflow para desenvolver
um algoritmo rápido, e depois melhorá-lo.

## Um pacote

Uma pacote em Julia, é um diretório com a configuração

 - NomeDoPacote.jl/
   - src/
     - NomeDoPacote.jl
     - Outros arquivos
   - test/
     - runtests.jl
     - Arquivos auxiliares de teste.
   - README.md
   - LICENSE.md
   - .travis.yml
   - Outros arquivos
   
Por mais incomum que seja, o nome do nosso pacote será o nome do diretório,
**incluindo o .jl**. Além disso, esse mesmo nome será o nome do arquivo
principal do pacote.

Os arquivos do `src/` sozinhos não fazem nada. Eles declaram e definem as
funções do pacote. O arquivo `test/` é quem chama o código, assim como
o usuário chamaria. O arquivo `runtests.jl` é o arquivo chamado quando o
usuário fizer `Pkg.test("NomeDoPacote")`.
No índice de pacotes é mostrado se esse teste é bem sucedido.

O arquivo `README.md` é o arquivo principal de informação do seu pacote,
e o arquivo `LICENSE.md` é o arquivo indicando qual a licensa do seu pacote.
A licensa mais utilizada aqui é a MIT, mas outras podem ser utilizadas.

O arquivo `.travis.yml` é um arquivo muito específico que roda os testes
do seu pacote online. Ele exige configuração, e não vamos falar dele agora.

## NomeDoPacote.jl

O arquivo `NomeDoPacote.jl` é o arquivo principal do seu pacote.
A maneira normal de começar é criar um módulo para seu código.

````julia
module NomeDoPacote

...

end
````

Dentro desse módulo, normalmente incluímos códigos auxiliares. Por exemplo, vamos colocar a busca linear num
arquivo separado `buscaLinear.jl`. Então colocamos `include("buscaLinear.jl")`.

Agora criamos a nossa função `metodo_gradiente` e colocamos um comando
`export metodo_gradiente` para que usuários possam usar essa função.

Agora criamos algum teste para nosso código. Vamos criar o arquivo
`teste_quad.jl` contendo testes de funções quadráticas, e o no arquivo
`runtests.jl` colocamos `include("teste_quad.jl")`.

Veja o MetodoGradiente.jl.

## Workflow de um código de otimização

Até bem pouco tempo, tínhamos duas opções de implementações
de algoritmos de otimização

 - C/Fortran: Um algoritmo rápido, competitivo, porém trabalhoso de fazer e usar;
 - MatLab: Um algoritmo fácil de escrever, porém limitado em alguns aspectos de velocidade e liberdade de uso.

O Python criou uma alternativa que não fui suficientemente adotada por não ser fácil o suficiente,
mas com Julia, temos uma oportunidade nova.

O Julia permite uma implementação rápida, é livre, e tem uma velocidade boa.
Além disse, permite uma integração com C e Fortran muito maior, que pode ser feita gradualmente.
Eventualmente, se necessário, pode-se fazer uma migração total.

Esse paradigma funciona para quase qualquer tipo de aplicação em Julia,
mas para otimização vamos ver como o CUTEst pode nos ajudar.

### Ponto inicial

A primeira preocupação deve ser fazer o algoritmo funcionar.

 - Não se preocupe em deixar o código eficiente;
 - Não se preocupe com sistemas lineares;
 - Não se preocupe com BLAS
 - Não se preocupe com variáveis repetidas;
 - Não se preocupe com matrizes criadas à toa;

Palavras do Donald Knuth:

> Programmers waste enormous amounts of time thinking about, or worrying about, the speed of noncritical parts of their programs, and these attempts at efficiency actually have a strong negative impact when debugging and maintenance are considered. We should forget about small efficiencies, say about 97% of the time: **premature optimization is the root of all evil.** Yet we should not pass up our opportunities in that critical 3%.
> - Structured Programming With Go To Statements", Computing Surveys, Vol 6, No 4, December 1974 

Escreva os testes, decida o que o código precisa resolver e implemente tudo.

 - Use chamadas de função explícitas;
 - Use `\` para resolver sistemas;

In [28]:
function metodo_bfgs(f, ∇f, x₀; ϵ = 1e-4, α = 0.5, η = 0.5, kmax = 1000)
    B = eye(length(x₀))
    ef = 0
    x = copy(x₀)
    k = 0
    while norm(∇f(x)) > ϵ
        d = -B\∇f(x)
        dt∇f = dot(d,∇f(x))
        t = 1.0
        while f(x + t*d) > f(x) + α*t*dt∇f
            t *= η
        end
        s = t*d
        y = ∇f(x+s)- ∇f(x)
        x = x + s
        
        k += 1
        if k > kmax
            ef = 1
            break
        end
        
        if dot(s,y) > 0.0
            B = B + y*y'/dot(y,s) - B*s*s'*B/dot(s,B*s)
        end
    end
    return x, f(x), ef, k
end

metodo_bfgs (generic function with 1 method)

### CUTEst

Seu código funciona? Teste com o CUTEst. Neste caso você pode usar as funções de nível usuário final ou nível intermediário.

In [29]:
# Nível usuário final
using CUTEst
nlp = CUTEstModel("ROSENBR")
f(x) = obj(nlp, x)
∇f(x) = grad(nlp, x)

x, fx, ef, k = metodo_bfgs(f, ∇f, nlp.meta.x0, ϵ=1e-5, kmax=100000)
println("x = $x, f(x) = $fx; ef = $ef; k = $k")
cutest_finalize(nlp)


 Problem name: ROSENBR

 Double precision version will be formed

 The objective function uses 1 linear group
 The objective function uses 1 nonlinear group
 
 There are 2 free variables
 
 
 File successfully decoded
x = [0.9999996948881412,0.9999993862916682], f(x) = 9.430756494150018e-14; ef = 0; k = 34


In [30]:
# Nível intermediário
using CUTEst
nlp = CUTEstModel("ROSENBR")
f(x) = ufn(nlp, x)
∇f(x) = ugr(nlp, x)

x, fx, ef, k = metodo_bfgs(f, ∇f, nlp.meta.x0, ϵ=1e-5, kmax=100000)
println("x = $x, f(x) = $fx; ef = $ef; k = $k")
cutest_finalize(nlp)


 Problem name: ROSENBR

 Double precision version will be formed

 The objective function uses 1 linear group
 The objective function uses 1 nonlinear group
 
 There are 2 free variables
 
 
 File successfully decoded
x = [0.9999996948881412,0.9999993862916682], f(x) = 9.430756494150018e-14; ef = 0; k = 34


#### Rodando para uma lista de problemas

In [31]:
lista = ["ROSENBR", "BARD", "HS1"]

for p in lista
    nlp = CUTEstModel(p)
    f(x) = ufn(nlp, x)
    ∇f(x) = ugr(nlp, x)
    x, fx, ef, k = metodo_bfgs(f, ∇f, nlp.meta.x0, ϵ=1e-5, kmax=1000)
    println("x = $x, f(x) = $fx; ef = $ef; k = $k")
    cutest_finalize(nlp)
end


 Problem name: ROSENBR

 Double precision version will be formed

 The objective function uses 1 linear group
 The objective function uses 1 nonlinear group
 
 There are 2 free variables
 
 
 File successfully decoded
x = [0.9999996948881412,0.9999993862916682], f(x) = 9.430756494150018e-14; ef = 0; k = 34

 Problem name: BARD

 Double precision version will be formed

 The objective function uses 15 nonlinear groups
 
 There are 3 free variables
 
 
 File successfully decoded
x = [0.08241305128208425,1.1331146993324164,2.3436197365623053], f(x) = 0.008214877350822292; ef = 0; k = 18

 Problem name: HS1

 Double precision version will be formed

 The objective function uses 2 nonlinear groups
 
 There is 1 free variable 
 There is 1 variable bounded only from below 
 
 
 File successfully decoded
x = [0.9999999998916259,0.9999999997869166], f(x) = 1.3087971232937582e-20; ef = 0; k = 43


### Melhorias

Agora que temos um código e sabemos que ele funciona para alguns problemas (a lista pode ser maior), vamos fazer algumas melhorias.

#### Exception Handling

Quase toda linguagem atual tem alguma maneira de lidar com exceções.
Como `nlp` precisa ser finalizado, podemos fazer o seguinte

In [32]:
lista = ["ROSENBR", "BARD", "HS1"]
times = zeros(length(lista))

for (i,p) in enumerate(lista)
    nlp = CUTEstModel(p)
    f(x) = ufn(nlp, x)
    ∇f(x) = ugr(nlp, x)
    try
        times[i] = time()
        x, fx, ef, k = metodo_bfgs(f, ∇f, nlp.meta.x0, ϵ=1e-5, kmax=1000)
        times[i] = time() - times[i]
        println("x = $x, f(x) = $fx; ef = $ef; k = $k")
    catch er
        println("ERRO no problema $p:", er)
    finally
        cutest_finalize(nlp)
    end
end
println(times)


 Problem name: ROSENBR

 Double precision version will be formed

 The objective function uses 1 linear group
 The objective function uses 1 nonlinear group
 
 There are 2 free variables
 
 
 File successfully decoded
x = [0.9999996948881412,0.9999993862916682], f(x) = 9.430756494150018e-14; ef = 0; k = 34

 Problem name: BARD

 Double precision version will be formed

 The objective function uses 15 nonlinear groups
 
 There are 3 free variables
 
 
 File successfully decoded
x = [0.08241305128208425,1.1331146993324164,2.3436197365623053], f(x) = 0.008214877350822292; ef = 0; k = 18

 Problem name: HS1

 Double precision version will be formed

 The objective function uses 2 nonlinear groups
 
 There is 1 free variable 
 There is 1 variable bounded only from below 
 
 
 File successfully decoded
x = [0.9999999998916259,0.9999999997869166], f(x) = 1.3087971232937582e-20; ef = 0; k = 43
[0.006665945053100586,0.0010790824890136719,0.0016868114471435547]


#### Reaproveitar

 - Chamar `f(x)` toda vez é ruim.
 - Produtos internos
 - Bs = B*s

In [33]:
function metodo_bfgs(f, ∇f, x₀; ϵ = 1e-4, α = 0.5, η = 0.5, kmax = 1000)
    B = eye(length(x₀))
    ef = 0
    x = copy(x₀)
    fx = f(x)
    ∇fx = ∇f(x)
    k = 0
    while norm(∇fx) > ϵ
        d = -B\∇fx
        dt∇f = dot(d,∇fx)
        t = 1.0
        while f(x + t*d) > fx + α*t*dt∇f
            t *= η
        end
        s = t*d
        x = x + s
        y = ∇fx # Não copia, só passa o ponteiro
        fx = f(x)
        ∇fx = ∇f(x)
        y = ∇fx - y
        
        k += 1
        if k > kmax
            ef = 1
            break
        end
        
        sty = dot(s,y)
        if sty > 0.0
            Bs = B*s
            B = B + y*y'/sty - Bs*Bs'/dot(s,Bs)
        end
    end
    return x, fx, ef, k
end

metodo_bfgs (generic function with 1 method)

In [34]:
lista = ["ROSENBR", "BARD", "HS1"]
times = zeros(length(lista))

for (i,p) in enumerate(lista)
    nlp = CUTEstModel(p)
    f(x) = ufn(nlp, x)
    ∇f(x) = ugr(nlp, x)
    try
        times[i] = time()
        x, fx, ef, k = metodo_bfgs(f, ∇f, nlp.meta.x0, ϵ=1e-5, kmax=1000)
        times[i] = time() - times[i]
        println("x = $x, f(x) = $fx; ef = $ef; k = $k")
    catch er
        println("ERRO no problema $p:", er)
    finally
        cutest_finalize(nlp)
    end
end
println(times)


 Problem name: ROSENBR

 Double precision version will be formed

 The objective function uses 1 linear group
 The objective function uses 1 nonlinear group
 
 There are 2 free variables
 
 
 File successfully decoded
x = [0.9999996948881872,0.9999993862917482], f(x) = 9.43075451728985e-14; ef = 0; k = 34

 Problem name: BARD

 Double precision version will be formed

 The objective function uses 15 nonlinear groups
 
 There are 3 free variables
 
 
 File successfully decoded
x = [0.08241305128208426,1.1331146993324168,2.3436197365623053], f(x) = 0.008214877350822289; ef = 0; k = 18

 Problem name: HS1

 Double precision version will be formed

 The objective function uses 2 nonlinear groups
 
 There is 1 free variable 
 There is 1 variable bounded only from below 
 
 
 File successfully decoded
x = [0.9999999998916259,0.9999999997869166], f(x) = 1.3087971232937582e-20; ef = 0; k = 43
[0.07417082786560059,0.0004680156707763672,0.0009219646453857422]
